In [3]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras.applications import VGG16
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import os

In [2]:
!pip install tensorflow


  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/80/6f/57d36f6507e432d7fc1956b2e9e8530c5c2d2bfcd8821bcbfae271cd6688/tensorflow-2.14.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for tensorflow-intel==2.14.0 from https://files.pythonhosted.org/packages/ad/6e/1bfe367855dd87467564f7bf9fa14f3b17889988e79598bc37bf18f5ffb6/tensorflow_intel-2.14.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for absl-py>=1.0.0 from https://files.pythonhosted.org/packages/01/e4/dc0a1dcc4e74e08d7abedab278c795eef54a224363bb18f5692f416d834f/absl_py-2.0.0-py3-none-any.whl.metadata
  Using cached absl_py-2.0.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Obtaining dependency information for flatbuffers>=23.5.26 from https://files.pythonhosted.org/packages/6f/12/d5c79ee252793ffe845d58a913197bfa02ae9a0b5c9bc3dc4b58d477b9e7/flatbuffers-23.5.26-py2.py3-none-any.whl.metada

In [5]:
train_dir = 'D:/21_classs_8k/train'
valid_dir = 'D:/21_classs_8k/validation'
test_dir = 'D:/21_classs_8k/test'
target_size = (224, 224)  # VGG16 input size

datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=target_size,
    batch_size=32,
    class_mode='categorical'
)

valid_generator = datagen.flow_from_directory(
    valid_dir,
    target_size=target_size,
    batch_size=32,\
    class_mode='categorical',
    shuffle=False
)

test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=target_size,
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

Found 5880 images belonging to 21 classes.
Found 840 images belonging to 21 classes.
Found 1680 images belonging to 21 classes.


In [7]:
class_names = sorted(os.listdir(train_dir))

In [8]:
number_class=len(class_names)
print("Number Of Classes: ",number_class)

Number Of Classes:  21


In [9]:
print("Classes name are : ",class_names)

Classes name are :  ['ইউটার্ন', 'ইউটার্ন নেওয়া নিষেধ', 'একদিকে চলাচল', 'ওভারটেকিং নিষেধ', 'ট্র্যাক প্রবেশ নিষেধ', 'ডানদিকে মোড় নেওয়া নিষেধ', 'থামানো নিষেধ', 'পার্কিং', 'পার্কিং নিষেধ', 'প্রবেশ নিষেধ', 'বামদিকে মোড় নেওয়া নিষেধ', 'বাস স্ট্যান্ড', 'মসজিদ', 'রিকশা চলাচল নিষেধ', 'সামনে ডানদিকে আচমকা মোড় আছে', 'সামনে পথচারী পারাপার', 'সামনে বামদিকে পার্শ্ব রাস্তা আছে', 'সামনে স্কুল', 'হর্ন বাজানো নিষেধ', 'হাসপাতাল', 'হুইলচেয়ার']


In [10]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


58889256/58889256 [==============================] - 33s 1us/step


In [11]:
for layer in base_model.layers:
    layer.trainable = False


In [12]:

from tensorflow.keras.regularizers import l2

model = Sequential()
model.add(base_model)  # Adding the VGG16 base model
model.add(Flatten())
model.add(Dense(256, activation='relu', kernel_regularizer=l2(0.01)))  # Adding L2 regularization
model.add(Dropout(0.5))  # Adding dropout regularization
model.add(Dense(len(class_names), activation='softmax'))



In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 256)               6422784   
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 21)                5397      
                                                                 
Total params: 21142869 (80.65 MB)
Trainable params: 6428181 (24.52 MB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


In [14]:
print(len(class_names))

21


In [15]:
from keras.optimizers import Adam
opt = Adam(learning_rate=0.0001)
model.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [16]:
from keras.callbacks import EarlyStopping,ModelCheckpoint
es=EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=20)

In [17]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.callbacks import ModelCheckpoint

# Define a callback to save the model with the best accuracy
mc = ModelCheckpoint('D:/images_data/best_model.h5', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)


In [18]:
num_sessions = 3
epochs_per_session = 5

for session in range(num_sessions):
    start_epoch = session * epochs_per_session + 1
    end_epoch = start_epoch + epochs_per_session - 1
    print(f"Session {session + 1}: epochs {start_epoch}-{end_epoch}")
    model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // train_generator.batch_size,
    epochs=5,
    validation_data=valid_generator,
    validation_steps=valid_generator.n // valid_generator.batch_size,callbacks=[mc,es])
    

Session 1: epochs 1-5
Epoch 1/5
183/183 [==============================] - ETA: 0s - loss: 4.0439 - accuracy: 0.7308
Epoch 1: val_accuracy improved from -inf to 0.98438, saving model to D:/images_data\best_model.h5


C:\ProgramData\anaconda3\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


183/183 [==============================] - 1629s 9s/step - loss: 4.0439 - accuracy: 0.7308 - val_loss: 2.1981 - val_accuracy: 0.9844
Epoch 2/5
183/183 [==============================] - ETA: 0s - loss: 1.8553 - accuracy: 0.9596
Epoch 2: val_accuracy improved from 0.98438 to 0.99399, saving model to D:/images_data\best_model.h5
183/183 [==============================] - 1625s 9s/step - loss: 1.8553 - accuracy: 0.9596 - val_loss: 1.4332 - val_accuracy: 0.9940
Epoch 3/5
183/183 [==============================] - ETA: 0s - loss: 1.2643 - accuracy: 0.9851
Epoch 3: val_accuracy improved from 0.99399 to 0.99519, saving model to D:/images_data\best_model.h5
183/183 [==============================] - 1634s 9s/step - loss: 1.2643 - accuracy: 0.9851 - val_loss: 1.0240 - val_accuracy: 0.9952
Epoch 4/5
183/183 [==============================] - ETA: 0s - loss: 0.9256 - accuracy: 0.9906
Epoch 4: val_accuracy did not improve from 0.99519
183/183 [==============================] - 1487s 8s/step - loss

In [19]:
# Assuming you have a test_generator for generating test data

# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.n // test_generator.batch_size)

print(f"Test Accuracy: {test_accuracy}")


52/52 [==============================] - 313s 6s/step - loss: 0.1919 - accuracy: 0.9952
Test Accuracy: 0.995192289352417


In [20]:
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.n // test_generator.batch_size)
print('Test accuracy:', test_acc)

52/52 [==============================] - 372s 7s/step - loss: 0.1919 - accuracy: 0.9952
Test accuracy: 0.995192289352417


In [19]:
predictions = model.predict(test_generator, steps=test_generator.n // test_generator.batch_size)

26/26 [==============================] - 173s 7s/step


In [22]:
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report, confusion_matrix

# Load the saved model
loaded_model = load_model('D:/images_data/best_model.h5')

# Compile the model (if not already compiled)
loaded_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Make predictions on the test data
predictions = loaded_model.predict(test_generator, steps=len(test_generator), verbose=1)

# Get the true labels from the test generator
true_labels = test_generator.classes

# Convert predicted probabilities to class labels
predicted_labels = predictions.argmax(axis=1)

# Calculate the classification report
report = classification_report(true_labels, predicted_labels, target_names=test_generator.class_indices)
print("Classification Report:\n", report)

# Calculate the confusion matrix
confusion = confusion_matrix(true_labels, predicted_labels)
print("Confusion Matrix:\n", confusion)


53/53 [==============================] - 361s 7s/step
Classification Report:
                                   precision    recall  f1-score   support

                         ইউটার্ন       1.00      1.00      1.00        80
            ইউটার্ন নেওয়া নিষেধ       1.00      1.00      1.00        80
                    একদিকে চলাচল       0.99      0.99      0.99        80
                 ওভারটেকিং নিষেধ       1.00      0.99      0.99        80
            ট্র্যাক প্রবেশ নিষেধ       1.00      1.00      1.00        80
       ডানদিকে মোড় নেওয়া নিষেধ       0.99      1.00      0.99        80
                    থামানো নিষেধ       1.00      1.00      1.00        80
                         পার্কিং       1.00      1.00      1.00        80
                   পার্কিং নিষেধ       1.00      0.99      0.99        80
                    প্রবেশ নিষেধ       1.00      1.00      1.00        80
       বামদিকে মোড় নেওয়া নিষেধ       1.00      1.00      1.00        80
                   বাস স্ট্যান্ড 

In [23]:
from sklearn.metrics import multilabel_confusion_matrix

# Calculate multilabel confusion matrices
multilabel_confusion_mtx = multilabel_confusion_matrix(true_labels, predicted_labels)

# Sum the individual matrices element-wise to create the overall confusion matrix
overall_confusion_mtx = multilabel_confusion_mtx.sum(axis=0)

# Print the overall confusion matrix
print("Overall Confusion Matrix:")
print(overall_confusion_mtx)

Overall Confusion Matrix:
[[33596     4]
 [    4  1676]]


In [27]:
import matplotlib.pyplot as plt
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator

# Load your saved model with its weights
model = load_model('D:/images_data/best_model.h5')

# Define the data generators for test and validation datasets
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

test_generator = test_datagen.flow_from_directory(
    'D:/21_classs_8k/test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',  # Adjust this based on your data
    shuffle=False
)

validation_generator = test_datagen.flow_from_directory(
    'D:/21_classs_8k/validation',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',  # Adjust this based on your data
    shuffle=False
)

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(test_generator)

# Print the test accuracy
print(f'Test Accuracy: {test_accuracy}')

# Plot accuracy and loss
history = model.history.history
plt.plot(history['accuracy'])  # Change 'accuracy' to 'acc'
plt.plot(history['val_accuracy'])  # Change 'val_accuracy' to 'val_acc'
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

plt.plot(history['loss'])
plt.plot(history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()


Found 1680 images belonging to 21 classes.
Found 840 images belonging to 21 classes.
53/53 [==============================] - 321s 6s/step - loss: 0.4750 - accuracy: 0.9976
Test Accuracy: 0.9976190328598022


KeyError: 'accuracy'

In [20]:
model.load_weights("H:/images_data/best_model.h5")

In [21]:
model_json = model.to_json()
with open("H:/images_data/model.json","w") as json_file:
  json_file.write(model_json)

In [22]:
from keras.models import model_from_json
def predict_(image_path):
    # Load the Model from Json File
    json_file = open('H:/images_data/model.json', 'r')
    model_json_c = json_file.read()
    json_file.close()
    model_c = model_from_json(model_json_c)
    # Load the weights
    model_c.load_weights("H:/images_data/best_model.h5")
    # Compile the model
    opt = SGD(lr=1e-4, momentum=0.9)
    model_c.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
    # Load the image you want to classify
    image = cv2.imread(image_path)
    image = cv2.resize(image, (224, 224))
    cv2_imshow(image)
    # Predict the image
    preds = model_c.predict(np.expand_dims(image, axis=0))[0]
    predicted_class_index = np.argmax(preds)  # Get the index of the predicted class
    # Assuming you have a list of class names
    class_names = ['ইউটার্ন', 'ইউটার্ন নেওয়া নিষেধ', 'একদিকে চলাচল', 'ওভারটেকিং নিষেধ', 
                   'ট্র্যাক প্রবেশ নিষেধ', 'ডানদিকে মোড় নেওয়া নিষেধ', 'থামানো নিষেধ', 
                   'পার্কিং', 'পার্কিং নিষেধ', 'প্রবেশ নিষেধ', 'বামদিকে মোড় নেওয়া নিষেধ', 
                   'বাস স্ট্যান্ড', 'মসজিদ', 'রিকশা চলাচল নিষেধ', 
                   'সামনে ডানদিকে আচমকা মোড় আছে', 'সামনে পথচারী পারাপার', 
                   'সামনে বামদিকে পার্শ্ব রাস্তা আছে', 'সামনে স্কুল', 'হর্ন বাজানো নিষেধ', 'হাসপাতাল','হুইলচেয়ার'] # Replace with your actual class names
    predicted_class_name = class_names[predicted_class_index]
    print("Predicted Label:", predicted_class_name)


In [23]:
import numpy as np
from PIL import Image
# Load the image
image_path ="H:\\image.png"
img = Image.open(image_path)
img = img.resize((224, 224))  # Resize the image to match the input size of VGG16

# Convert the image to RGB mode
img = img.convert("RGB")

# Preprocess the image
img_array = np.array(img)
img_array = img_array / 255.0  # Normalize the pixel values
img_array = np.expand_dims(img_array, axis=0)  # Add a batch dimension

# Make the prediction
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions)

print("Predicted class:", class_names[predicted_class])


1/1 [==============================] - 0s 330ms/step
Predicted class: ইউটার্ন


In [2]:


# Load the Keras model
model = load_model("H:/images_data/best_model.h5")

# Convert the model to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TensorFlow Lite model to a file
with open("best_model.tflite", "wb") as f:
    f.write(tflite_model)



INFO:tensorflow:Assets written to: C:\Users\TANJIL\AppData\Local\Temp\tmpfouxoy15\assets


INFO:tensorflow:Assets written to: C:\Users\TANJIL\AppData\Local\Temp\tmpfouxoy15\assets


###### 

In [22]:
import numpy as np
from PIL import Image

# Load the image
image_path = "H:\Raw image from internet\\6137316-no-car-parking-sign-isolated-over-white.webp"
img = Image.open(image_path)
img = img.resize((224, 224))  # Resize the image to match the input size of VGG16

# Convert the image to grayscale
img = img.convert("L")

# Preprocess the image
img_array = np.array(img)
img_array = img_array / 255.0  # Normalize the pixel values
img_array = np.expand_dims(img_array, axis=-1)  # Add a channel dimension
img_array = np.repeat(img_array, 3, axis=-1)  # Repeat the grayscale image across channels
img_array = np.expand_dims(img_array, axis=0)  # Add a batch dimension

# Make the prediction
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions)

print("Predicted class:", class_names[predicted_class])


1/1 [==============================] - 0s 306ms/step
Predicted class: হর্ন বাজানো নিষেধ


In [23]:
import numpy as np
from PIL import Image

# Load the RGB image
image_path = "H:\Raw image from internet\OVERTALIKNG.jpg"
img = Image.open(image_path)
img = img.resize((224, 224))  # Resize the image to match the input size of VGG16

# Convert the image to grayscale
img_gray = img.convert("L")

# Preprocess the image
img_array = np.array(img_gray)
img_array = img_array / 255.0  # Normalize the pixel values
img_array = np.expand_dims(img_array, axis=0)  # Add a batch dimension

# Make the prediction
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions)

print("Predicted class:", class_names[predicted_class])


InvalidArgumentError: Graph execution error:

Detected at node 'sequential/vgg16/block1_conv1/Relu' defined at (most recent call last):
    File "C:\ProgramData\anaconda3\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\ProgramData\anaconda3\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\ProgramData\anaconda3\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\ProgramData\anaconda3\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
      app.start()
    File "C:\ProgramData\anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 711, in start
      self.io_loop.start()
    File "C:\ProgramData\anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "C:\ProgramData\anaconda3\lib\asyncio\base_events.py", line 603, in run_forever
      self._r